### Μαυρουδής Παναγιώτης tem3062 εργστήριο3 
- Μαζί με το παρρόν notebook επισυνάπτω ενα αρχείο που περιέχει το παραχθέν μοντέλο και ένα αρχείο με τις εκδώσεις των βιβλιοθηκών που χρησιμοποίησα.
- Αν αντιμετωπίσετε οποιοδήποτε θέμα αναφορικά με τις βιβλιοθήκες που χρησιμοποίησα μπορείτε να χρησιμοποιήσετε την εντολή pip install -r Spam_or_Ham_requirements.txt η οποία θα κατεβάσει όλες τις βιβλιοθήκες που σας λείπουν στο συστημά σας.

In [ ]:
import sys ,pickle

import pandas as pd 
import numpy as np 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

from imblearn.over_sampling import SMOTE

In [34]:
# ! pip install imbalanced-learn
# Download NLTK resources
# nltk.download('punkt')
# nltk.download('stopwords')

Defaulting to user installation because normal site-packages is not writeable


Some examining methods for python classes:

In [3]:
# import inspect

# # predictions = spam_inst.predict(test_corpus) 
# print(dir(spam_inst), "\n")    
# help(spam_inst)

# # List all methods of an object
# methods = [method for method in dir(spam_inst) if callable(getattr(spam_inst, method))]
# # print(methods)

# # Or to get more detailed information including the docstring
# for method in methods:
#     print(method)
#     print(inspect.getdoc(getattr(spam_inst, method)))

The main class 

In [43]:
class spam_detection:
    def __init__(self):
        self.model = None 

        self.vectorizer_path = 'fitted_vectorizer.pkl'
        self.vectorizer = None

        self.smote = SMOTE(random_state=42)
        
    def load_data(self, file):
        # Assuming the data is tab-separated and has no header
        data = pd.read_csv(file, sep='\t', header=None, names=['Label', 'Message'])
        return data 
    
    # Function to preprocess text
    def preprocess_text(self, message):
        """
        Preprocess a single SMS message by converting to lowercase, removing stopwords and non-alphanumeric characters, and tokenizing.

        Parameters:
        - message (str): The SMS message text.

        Returns:
        - str: The cleaned and preprocessed text.
        """
        # Convert to lowercase
        # sys.exit() 
        message = message.lower()
        # Tokenize
        words = word_tokenize(message)
        # Remove stopwords and punctuation
        filtered_words = [word for word in words if word not in stopwords.words('english') and word.isalnum()]
        return ' '.join(filtered_words)
    
    def fit_vectorizer(self, data):
        """
        Fit the vectorizer on the training data and save it to a file.
        
        Parameters:
        - data (pd.Series or list): Text data to fit the vectorizer.

        Returns:
        - The fitted vectorizer instance.
        """
        # Fit the vectorizer on the provided data
        self.vectorizer = TfidfVectorizer()
        self.vectorizer.fit(data)
        with open(self.vectorizer_path, 'wb') as file:
            pickle.dump(self.vectorizer, file)

    def load_vectorizer(self):
        """
        Load the pre-fitted vectorizer.
        """
        with open(self.vectorizer_path, 'rb') as file:
            self.vectorizer = pickle.load(file)
        return self.vectorizer

    def vectorize_text(self, data):
        """
        Convert text data into TF-IDF numeric format using a pre-fitted vectorizer loaded from a file.
        
        Parameters:
        - data (pd.Series or list): Text data to transform.

        Returns:
        - scipy.sparse.csr.csr_matrix: The TF-IDF matrix of the text data.
        """
        if not self.vectorizer:
            self.load_vectorizer()
        return self.vectorizer.transform(data)
   
    
    def preprocess_pipline(self, file, fit_vectorizer=False):
        """
        Preprocess the data by loading, cleaning, transforming, vectorizing the text messages, and applying SMOTE for class balancing.

        This function loads data from a specified file, preprocesses the text content of the messages, maps labels from text to integers, converts the processed text messages into a sparse matrix format suitable for machine learning models, and balances the classes using SMOTE.

        Parameters:
        - file (str): The path to the file containing the dataset. The file is expected to be in a format readable by `pandas` (e.g., CSV) with at least two columns: 'Message' and 'Label'.
        - fit_vectorizer (bool): Flag to determine if the vectorizer should be fit to the data or if an existing vectorizer should be used.

        Returns:
        - tuple:
            - X_resampled (scipy.sparse.csr.csr_matrix or numpy.ndarray): The resampled feature matrix after applying SMOTE, potentially dense if the number of features is large.
            - y_resampled (numpy.ndarray): The resampled label array with numerical values where 'spam' is 0 and 'ham' is 1.

        The function performs the following operations:
        1. Loads the data from the file using a custom `load_data` method.
        2. Iterates through each message, cleans, and preprocesses it using the `preprocess_text` method.
        3. Constructs a new DataFrame with preprocessed messages.
        4. Resets the index of the DataFrame to ensure it starts from 0.
        5. Maps textual labels ('spam', 'ham') to numerical values (0, 1) for model compatibility.
        6. Vectorizes the preprocessed text messages into a sparse matrix format using the `vectorize_text` method.
        7. Applies SMOTE to the vectorized text data to balance the classes, transforming the data matrix to handle class imbalance effectively.
        8. Outputs diagnostic information about the processed and resampled data, including the number of non-zero values and the shape of the matrix.

        Example usage:
        >>> spam_detector = SpamDetector()  # Assuming this method is part of the SpamDetector class
        >>> file_path = 'data/spam_data.csv'
        >>> X_resampled, y_resampled = spam_detector.preprocess_pipline(file_path, fit_vectorizer=True)
        """
        data = self.load_data(file)

        messages = data["Message"] 
        Filtered_messages = []
        for message in messages:
            filtered_words = self.preprocess_text(message)  
        
            # print(filtered_words) 
            Filtered_messages.append(filtered_words) 
            # print(Filtered_messages) 

        # Create new database with the preprocessed messages 
        data["Message"] = Filtered_messages 
        # (x_train["Label"] == "ham").sum() 
        # We have 611 spam Labels and 3846 ham messages.
        # Assuming df is your DataFrame
        data.reset_index(drop=True, inplace=True)

        # We define our mapping
        label_mapping = {'spam': 0,'ham': 1}

        # Apply the mapping to the column
        data['Label'] = data['Label'].map(label_mapping)
        Labels = data["Label"] 

        if fit_vectorizer:
            self.fit_vectorizer(data['Message'])

        corpus = self.vectorize_text(data["Message"]) 
        print(f"Non zero values = {corpus.nnz}\nShape of the Sparce matrix = {corpus.shape} ")  
        
        # Apply SMOTE to balance classes
        X_resampled, y_resampled = self.smote.fit_resample(corpus, data['Label'])

        return X_resampled, y_resampled

    def train_model(self, features, labels):
        """
        Train a Naive Bayes classifier using the provided features and labels.

        Parameters:
        - features (scipy.sparse.csr.csr_matrix or np.ndarray): The features for training.
        - labels (pd.Series or np.ndarray): The labels for training.
        """
        self.model = MultinomialNB()
        self.model.fit(features, labels)      
        
    def my_predict(self, text):
        """Predict whether a new text is spam or ham.
        
        Parameters:
        - text (str): The text message to classify.

        Returns:
        - prediction (array): Predicted label for the text.
        """
        if self.model is not None:
            # Ensure that text is in a list so it's treated as a single sample
            features = self.vectorize_text([text])
            return self.model.predict(features)
        else:
            print("Model has not been trained yet.")
            return None

    def evaluate_model(self, features, labels):
        """
        Evaluate the trained model using the provided test dataset and print the classification report.

        Parameters:
        - features (scipy.sparse.csr.csr_matrix or np.ndarray): The test features.
        - labels (pd.Series or np.ndarray): The test labels.
        """
        if self.model is not None:
            predictions = self.model.predict(features)
            print(classification_report(labels, predictions))
        else:
            print("Model is not trained or test data is not available.")
            
    def save_model(self, file_name='model.pkl'):
        """
        Save the fitted vectorizer and trained model to disk using pickle.

        This method serializes the vectorizer and classifier model to a file, allowing them to be loaded and used later without the need for retraining. This is particularly useful for deployment or for saving model checkpoints during training. By default, the model is saved to a file named 'model.pkl', but a custom file name can be specified.

        Parameters:
        - file_name (str): The name of the file to save the model to. Default is 'model.pkl'.

        Returns:
        - None
        """
        with open(file_name, 'wb') as f:
            pickle.dump((self.vectorizer, self.model), f) 


spam_inst = spam_detection()

In [44]:
x_train_file  = "spam_train.txt"
x_test_file = "spam_test.txt"
print("x_train")
train_corpus, train_labels = spam_inst.preprocess_pipline(x_train_file, fit_vectorizer=True)
print("x_test") 
test_corpus, test_labels = spam_inst.preprocess_pipline(x_test_file)

x_train
Non zero values = 36270
Shape of the Sparce matrix = (4457, 7186) 
x_test
Non zero values = 3988
Shape of the Sparce matrix = (558, 7186) 


In [45]:
print(f"Non zero values = {train_corpus.nnz}\nShape of the Sparce matrix = {train_corpus.shape} ")  
print(f"Non zero values = {test_corpus.nnz}\nShape of the Sparce matrix = {test_corpus.shape} ")   

Non zero values = 105821
Shape of the Sparce matrix = (7692, 7186) 
Non zero values = 13484
Shape of the Sparce matrix = (982, 7186) 


Only 36270 elements are non zero from a total of 32.028.002

In [47]:
# train the model 
spam_inst.train_model(train_corpus, train_labels)     
spam_inst.save_model()       

In [48]:
spam_inst.evaluate_model(test_corpus, test_labels)  

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       491
           1       0.98      0.98      0.98       491

    accuracy                           0.98       982
   macro avg       0.98      0.98      0.98       982
weighted avg       0.98      0.98      0.98       982



### Interpretation of the Classification Report

#### Basic Definitions:
- **Precision**: The ratio of correctly predicted positive observations to the total predicted positives. It shows how relevant the model's predictions are.
- **Recall** (Sensitivity or True Positive Rate): The ratio of correctly predicted positive observations to all actual positives. It shows how well the model can find all the positive samples.
- **F1-Score**: The weighted average of Precision and Recall. This score takes both false positives and false negatives into account. It is especially useful when the class distribution is uneven. The formula for the F1 score is:
  
  \[
  F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
  \]

- **Support**: The number of actual occurrences of the class in the specified dataset.

#### Results Explained:
- **Class 0 (Spam, assumed) Results**:
  - **Precision**: 1.00 - When your model predicts an instance as Class 0, it is correct every time.
  - **Recall**: 0.79 - However, the model only successfully identifies 79% of all actual Class 0 instances.
  - **F1-Score**: 0.88 - This is a measure of the test's accuracy. The score is relatively high, indicating a good balance between precision and recall.

- **Class 1 (Ham, assumed) Results**:
  - **Precision**: 0.97 - When your model predicts an instance as Class 1, it is correct 97% of the time.
  - **Recall**: 1.00 - The model successfully identifies 100% of all actual Class 1 instances.
  - **F1-Score**: 0.99 - A nearly perfect F1 score, indicating excellent precision and recall balance.

#### Overall Accuracy:
- **Accuracy**: 0.97 - Overall, the model correctly predicts the class of an instance 97% of the time across the total dataset of 558 instances.

#### Averages:
- **Macro Avg**: 
  - **Precision**: 0.99 
  - **Recall**: 0.90 
  - **F1-Score**: 0.93 - Macro-average will compute the metric independently for each class and then take the average (hence treating all classes equally).
- **Weighted Avg**: 
  - **Precision**: 0.98 
  - **Recall**: 0.97 
  - **F1-Score**: 0.97 - Weighted average takes into account the support of each class. This is useful when dealing with class imbalance as it gives a more representative picture of the overall performance.


In [65]:
test_messages = ["Free money now!!!", "ok see you there!", "Congratulations prize! Clic lt ok here."]
test_labels = [0, 1, 0] 

for msg, label in zip(test_messages, test_labels):
    prediction = spam_inst.my_predict(msg)
    print(f"Message: '{msg}' - Predicted: {'Spam' if prediction == 0 else 'Ham'}, Actual: {'Spam' if label == 0 else 'Ham'}")

Message: 'Free money now!!!' - Predicted: Spam, Actual: Spam
Message: 'ok see you there!' - Predicted: Ham, Actual: Ham
Message: 'Congratulations prize! Clic lt ok here.' - Predicted: Spam, Actual: Spam


## We need to find the 5 most important for the classification process words.

### Analysis of Word Significance in Email Classification

One way to determine how indicative a word $i$ is in an email classification context is to calculate the following ratio:

$$
\text{Odds Ratio} = \log \left(\frac{P(x_j = i \mid y = 1)}{P(x_j = i \mid y = 0)}\right)
$$

Where:
- $P(x_j = i \mid y = 1)$ is the probability of word $i$ occurring in an email classified as SPAM.
- $P(x_j = i \mid y = 0)$ is the probability of word $i$ occurring in an email classified as NOT SPAM.



In [49]:
log_prob_spam = spam_inst.model.feature_log_prob_[0]  # Log probabilities of words for the spam class
log_prob_ham = spam_inst.model.feature_log_prob_[1]   # Log probabilities of words for the non-spam class

# Calculate the difference in log probabilities
log_odds_ratio = log_prob_spam - log_prob_ham

In [50]:
feature_names = spam_inst.vectorizer.get_feature_names_out()

# Find the top five most indicative words for spam
top_spam_words_indices = np.argsort(log_odds_ratio)[-5:]  # Sort and get the top 5 indices
top_spam_words = [(feature_names[i], log_odds_ratio[i]) for i in top_spam_words_indices]

In [51]:
# Print the top five most indicative words for spam
print("Top indicative words for SPAM:")
for word, score in top_spam_words:
    print(f"{word}: {score}")

Top indicative words for SPAM:
awarded: 3.7793494522831006
tone: 3.806624049179675
guaranteed: 4.096346195280912
prize: 4.573297002722405
claim: 4.582111678041463


### Top Indicative Words for Ham
- **later**: 3.1176
- **come**: 3.1273
- **ok**: 3.1494
- **gt**: 3.3706
- **lt**: 3.3747


# Training SVM with RBF Kernel

In [52]:
# Initialize the SVM classifier with the RBF kernel
svm_classifier = SVC(kernel='rbf', gamma='scale')  
svm_classifier.fit(train_corpus, train_labels)

# Predict on the test set
y_pred = svm_classifier.predict(train_corpus)

# Evaluate the classifier
print(classification_report(train_labels, y_pred))
print("Accuracy:", accuracy_score(train_labels, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3846
           1       1.00      1.00      1.00      3846

    accuracy                           1.00      7692
   macro avg       1.00      1.00      1.00      7692
weighted avg       1.00      1.00      1.00      7692

Accuracy: 1.0


In [68]:
test_messages = [spam_inst.preprocess_text("Free money now!!!"),spam_inst.preprocess_text("Hello"), spam_inst.preprocess_text("Congratulations prize! Click here.")]
test_messages = spam_inst.vectorize_text(test_messages) 
test_labels = [0, 1, 0]  

for msg, label in zip(test_messages, test_labels):

    prediction = svm_classifier.predict(msg)
    print(f"Message: '{msg}' - Predicted: {'Spam' if prediction == 0 else 'Ham'}, Actual: {'Spam' if label == 0 else 'Ham'}")

Message: '  (0, 2737)	0.5925943429343582
  (0, 4231)	0.8055010519683983' - Predicted: Spam, Actual: Spam
Message: '  (0, 3125)	1.0' - Predicted: Ham, Actual: Ham
Message: '  (0, 1643)	0.6547779346195032
  (0, 1754)	0.6004185504149558
  (0, 5026)	0.45908977406714363' - Predicted: Ham, Actual: Spam


In [69]:
# Possible gamma values
gamma_values = np.logspace(-3, 2, 6)  # Creates an array of gamma values from 0.001 to 100 on a logarithmic scale
performance_scores = []

for gamma in gamma_values:
    svm_classifier = SVC(kernel='rbf', gamma=gamma)
    svm_classifier.fit(train_corpus, train_labels) 
    y_pred = svm_classifier.predict(train_corpus)
    accuracy = accuracy_score(train_labels, y_pred)
    performance_scores.append(accuracy)
    print(f"Gamma: {gamma}, Accuracy: {accuracy}")

# Finding the gamma value with the highest accuracy
optimal_gamma = gamma_values[np.argmax(performance_scores)]
print(f"Optimal Gamma: {optimal_gamma}")

Gamma: 0.001, Accuracy: 0.8619344773790951
Gamma: 0.01, Accuracy: 0.9418876755070202
Gamma: 0.1, Accuracy: 0.9914196567862714
Gamma: 1.0, Accuracy: 1.0
Gamma: 10.0, Accuracy: 1.0
Gamma: 100.0, Accuracy: 1.0
Optimal Gamma: 1.0
